### Apogee Prediction Model V1

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib  # for saving scaler

In [2]:
# Load the dataset
df = pd.read_csv("sliding_train_by_flight.csv")

# Fill NaNs with 0
df.fillna(0, inplace=True)

# Count initial number of columns
initial_cols = df.shape[1]

# Drop columns where all values are exactly 0
df = df.loc[:, (df != 0).any(axis=0)]

# Count and report how many were removed
removed_cols = initial_cols - df.shape[1]
print(f"Removed {removed_cols} all-zero columns.")


Removed 0 all-zero columns.


In [3]:
# Split features and target
X = df.drop(columns=["Apogee"]).values
y = df["Apogee"].values.reshape(-1, 1)

# Check for NaNs/infs just in case
assert not np.isnan(X).any(), "NaN found in features"
assert not np.isnan(y).any(), "NaN found in targets"

# Feature scaling
input_scaler = StandardScaler()
X_scaled = input_scaler.fit_transform(X)
joblib.dump(input_scaler, "apogee_input_scaler.pkl")

# Target scaling (important!)
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y)
joblib.dump(target_scaler, "apogee_target_scaler.pkl")

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)

In [4]:
# === Define MLP model ===
class ApogeeMLP(nn.Module):
    def __init__(self, input_dim):
        super(ApogeeMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

model = ApogeeMLP(X_train.shape[1]).to(device)

In [5]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
batch_size = 64

In [6]:
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))
    epoch_loss = 0

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x = X_train[indices]
        batch_y = y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val).item()

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1/50, Train Loss: 127.3676, Val Loss: 0.9385
Epoch 2/50, Train Loss: 104.9909, Val Loss: 0.6738
Epoch 3/50, Train Loss: 72.5778, Val Loss: 0.5224
Epoch 4/50, Train Loss: 50.9964, Val Loss: 0.3169
Epoch 5/50, Train Loss: 37.6936, Val Loss: 0.3764
Epoch 6/50, Train Loss: 35.7248, Val Loss: 0.2429
Epoch 7/50, Train Loss: 27.7878, Val Loss: 0.2725
Epoch 8/50, Train Loss: 32.1057, Val Loss: 0.1700
Epoch 9/50, Train Loss: 23.8943, Val Loss: 0.1654
Epoch 10/50, Train Loss: 21.9234, Val Loss: 0.1503
Epoch 11/50, Train Loss: 25.1119, Val Loss: 0.2258
Epoch 12/50, Train Loss: 19.5010, Val Loss: 0.1332
Epoch 13/50, Train Loss: 24.5451, Val Loss: 0.1739
Epoch 14/50, Train Loss: 18.9275, Val Loss: 0.1797
Epoch 15/50, Train Loss: 17.5464, Val Loss: 0.1244
Epoch 16/50, Train Loss: 18.5594, Val Loss: 0.1920
Epoch 17/50, Train Loss: 20.5375, Val Loss: 0.1716
Epoch 18/50, Train Loss: 18.3850, Val Loss: 0.1239
Epoch 19/50, Train Loss: 21.7959, Val Loss: 0.1336
Epoch 20/50, Train Loss: 17.5710, Val 

In [7]:
# Save the model
torch.save(model.state_dict(), "apogee_mlp_model.pth")
print("✅ Model and scalers saved (apogee_mlp_model.pth, apogee_input_scaler.pkl, apogee_target_scaler.pkl)")


✅ Model and scalers saved (apogee_mlp_model.pth, apogee_input_scaler.pkl, apogee_target_scaler.pkl)
